In [89]:
import numpy as np
import pandas as pd        
import tensorflow.compat.v1 as tf 
import matplotlib.pyplot as plt
tf.disable_v2_behavior()
from sklearn.model_selection import train_test_split

In [90]:
in_train_df = pd.read_hdf('df_train_in.h5', key='df')
in_train_matrix = in_train_df[['Month', 'Day', 'Weekday', 'Hours', 'Minutes', 'Temp', 'Precip', '-10']].as_matrix()
x1_train_matrix = in_train_df[['-5']].as_matrix()
x2_train_matrix = in_train_df[['Now']].as_matrix()
out_train_matrix = pd.read_hdf('df_train_out.h5', key = 'df').as_matrix()
in_eval_df = pd.read_hdf('df_evalu_in.h5', key='df')
in_eval_matrix = in_eval_df[['Month', 'Day', 'Weekday', 'Hours', 'Minutes', 'Temp', 'Precip', '-10']].as_matrix()
x1_eval_matrix = in_eval_df[['-5']].as_matrix()
x2_eval_matrix = in_eval_df[['Now']].as_matrix()
out_eval_matrix = pd.read_hdf('df_evalu_out.h5', key = 'df').as_matrix()

c:\users\janfd\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
c:\users\janfd\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\janfd\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
c:\users\janfd\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
c:\users\janfd\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Metho

In [91]:
in_train, in_test, out_train, out_test, x1_train, x1_test, x2_train, x2_test = train_test_split(
    in_train_matrix, out_train_matrix, x1_train_matrix, x2_train_matrix, test_size=0.2)

In [92]:
# Python optimisation variables
epochs = 25 #25
batch_size = 100

# Define Input and Output
inp = tf.placeholder(tf.float32, [None, 8], name='input')
out = tf.placeholder(tf.float32, [None, 1], name='Output')
x1 = tf.placeholder(tf.float32, [None, 1], name='x1')
x2 = tf.placeholder(tf.float32, [None, 1], name='x2')
const_1 = tf.constant(1.0, name="const_1")

In [93]:
# Scale Layer
w0 = tf.Variable(tf.random_normal([8, 10], stddev=0.03), name='w0')
b0 = tf.Variable(tf.random_normal([10]), name='b0')

y0 = tf.add(tf.matmul(inp, w0), b0)

# First Layer
wr1 = tf.Variable(tf.random_normal([10, 10], stddev=0.03), name='wr1')
wh1 = tf.Variable(tf.random_normal([10, 10], stddev=0.03), name='wh1')
wz1 = tf.Variable(tf.random_normal([10, 10], stddev=0.03), name='wz1')
br1 = tf.Variable(tf.random_normal([1, 10]), name='br1')
bh1 = tf.Variable(tf.random_normal([1, 10]), name='bh1')
bz1 = tf.Variable(tf.random_normal([1, 10]), name='bz1')

# r = logsigmoid(Br*x1 + Wr*inp)
r1 = tf.log_sigmoid(tf.add(tf.matmul(x1, br1), tf.matmul(y0, wr1)))
# H = tanh(Bh*x1 + Wh(r*inp)
H1 = tf.tanh(tf.add(tf.matmul(x1, bh1), tf.matmul(tf.multiply(y0, r1), wh1)))
# z = logsigmoid(Bz*x1 + Wz*inp)
z1 = tf.log_sigmoid(tf.add(tf.matmul(x1, bz1), tf.matmul(y0, wz1)))
# y1=(1 - z)inp + z*H
y1 = tf.add(tf.multiply(y0, tf.subtract(const_1, z1)), tf.multiply(H1, z1))

# Second Layer
wr2 = tf.Variable(tf.random_normal([10, 10], stddev=0.03), name='wr2')
wh2 = tf.Variable(tf.random_normal([10, 10], stddev=0.03), name='wh2')
wz2 = tf.Variable(tf.random_normal([10, 10], stddev=0.03), name='wz2')
br2 = tf.Variable(tf.random_normal([1, 10]), name='br2')
bh2 = tf.Variable(tf.random_normal([1, 10]), name='bh2')
bz2 = tf.Variable(tf.random_normal([1, 10]), name='bz2')

# r = logsigmoid(Br*x1 + Wr*inp)
r2 = tf.log_sigmoid(tf.add(tf.matmul(x2, br2), tf.matmul(y1, wr2)))
# H = tanh(Bh*x1 + Wh(r*inp)
H2 = tf.tanh(tf.add(tf.matmul(x2, bh2), tf.matmul(tf.multiply(y1, r2), wh2)))
# z = logsigmoid(Bz*x1 + Wz*inp)
z2 = tf.log_sigmoid(tf.add(tf.matmul(x2, bz2), tf.matmul(y1, wz2)))
# y1=(1 - z)inp + z*H
y2 = tf.add(tf.multiply(y1, tf.subtract(const_1, z2)), tf.multiply(H2, z2))

# Output Layer
wo = tf.Variable(tf.random_normal([10, 1], stddev=0.03), name='wo')
bo = tf.Variable(tf.random_normal([1]), name='bo')

yo = tf.add(tf.matmul(y2, wo), bo)

In [94]:
# Loss Function
cross_entropy = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(out, yo)))) #RMSE
# cross_entropy = tf.reduce_mean(tf.square(out - yo)) #MSE

# Optimizer
# optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
optimiser = tf.train.AdamOptimizer().minimize(cross_entropy)


In [95]:
# Initialisation Operator
init_op = tf.global_variables_initializer()

# Accuracy (same as Loss)
# correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(yo, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# accuracy = tf.reduce_mean(tf.cast(tf.abs(tf.subtract(yo, out)), tf.float32))
# accuracy = tf.reduce_mean(tf.cast(tf.subtract(constant_one, tf.abs(tf.subtract(yo, out))), tf.float32))
# accuracy =  tf.reduce_mean(tf.abs(tf.cast(tf.subtract(constant_one,tf.sqrt(tf.reduce_mean(
#     tf.square(tf.subtract(out, yo))))), tf.float32)))
accuracy =  (tf.subtract(const_1,tf.sqrt(tf.reduce_mean(tf.square(
    tf.subtract(out, tf.clip_by_value(yo, 0, 1)))))))


In [96]:
# accuracy_data = np.empty(epochs)
# 
# # Start Session
# with tf.Session() as sess:
#     # initialise the variables
#     sess.run(init_op)
#     total_batch = int(len(in_train) / batch_size) #in_train
#     acc_train = accuracy.eval(session= sess, feed_dict={inp: in_train, out: out_train, x1: x1_train, x2: x2_train})
#     print("Beginning accuracy: {:3.2f}%".format(acc_train*100.0))
#     for epoch in range(epochs):
#         avg_cost = 0
#         for i in range(total_batch):
#             batch_in = in_train[i*batch_size:(i+1)*batch_size]
#             batch_out = out_train[i*batch_size:(i+1)*batch_size]
#             batch_x1 = x1_train[i*batch_size:(i+1)*batch_size]
#             batch_x2 = x2_train[i*batch_size:(i+1)*batch_size]
# 
#             _, c = sess.run([optimiser, cross_entropy], 
#                             feed_dict={inp: batch_in, out: batch_out, x1: batch_x1, x2: batch_x2})
# 
#             avg_cost += c / total_batch
#         print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
# 
#         # Save Accuracy over Iterations
#         acc_train = accuracy.eval(session= sess, feed_dict={inp: in_train, out: out_train, x1: x1_train, x2: x2_train})
#         print("Accuracy: {:3.2f}%".format(acc_train*100.0))
#         accuracy_data[epoch] = acc_train
# 
#     # Calculate Accuracy on Train and Test
#     acc_train = accuracy.eval(session= sess, feed_dict={inp: in_train, out: out_train, x1: x1_train, x2: x2_train})
#     print("Train accuracy: {:3.2f}%".format(acc_train*100.0))
# 
#     acc_test = accuracy.eval(session= sess, feed_dict={inp: in_test, out: out_test, x1: x1_test, x2: x2_test})
#     print("Test accuracy:  {:3.2f}%".format(acc_test*100.0))
# 
#     # Evaluate
#     acc_eval = accuracy.eval(session= sess, feed_dict={inp: in_eval_matrix, out: out_eval_matrix, x1: x1_eval_matrix, x2: x2_eval_matrix})
#     print("Eval accuracy:  {:3.2f}%".format(acc_eval*100.0))    
# 
#     # Save Dataframe with Accuracy over Iterations
#     acc_iter = pd.DataFrame({'Accuracy': accuracy_data}).to_hdf('df_acc_gru.h5',key='df', mode='w')
# 
#     # Save Dataframe with Predictions for Evaluation Dataset
#     predictions_raw = sess.run(yo,feed_dict={inp: in_eval_matrix, out: out_eval_matrix, x1: x1_eval_matrix, x2: x2_eval_matrix})
#     predictions = pd.concat([pd.DataFrame(out_eval_matrix), pd.DataFrame(predictions_raw)], axis=1)
#     predictions.columns = ['Output', 'Prediction']
#     predictions['Prediction'].to_hdf('df_evalu_pred_gru.h5',key='df', mode='w')
#     

In [97]:
# Loop to get a mean Accuracy
acc_all = 0.0
acc_iter = pd.DataFrame(index=range(epochs),columns=['Accuracy']).fillna(value=0)
n = 20 #20
for mean_iter in range(n):

    accuracy_data = np.empty(epochs)

    # Start Session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(in_train) / batch_size) #in_train
        # acc_train = accuracy.eval(session= sess, feed_dict={inp: in_train, out: out_train, x1: x1_train, x2: x2_train})
        # print("Beginning accuracy: {:3.2f}%".format(acc_train*100.0))
        for epoch in range(epochs):
            avg_cost = 0
            for i in range(total_batch):
                batch_in = in_train[i*batch_size:(i+1)*batch_size]
                batch_out = out_train[i*batch_size:(i+1)*batch_size]
                batch_x1 = x1_train[i*batch_size:(i+1)*batch_size]
                batch_x2 = x2_train[i*batch_size:(i+1)*batch_size]

                _, c = sess.run([optimiser, cross_entropy], 
                                feed_dict={inp: batch_in, out: batch_out, x1: batch_x1, x2: batch_x2})

            #     avg_cost += c / total_batch
            # print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))

            # Save Accuracy over Iterations
            acc_train = accuracy.eval(session= sess, feed_dict={inp: in_train, out: out_train, x1: x1_train, x2: x2_train})
            # print("Accuracy: {:3.2f}%".format(acc_train*100.0))
            accuracy_data[epoch] = acc_train

        # # Calculate Accuracy on Train and Test
        # acc_train = accuracy.eval(session= sess, feed_dict={inp: in_train, out: out_train, x1: x1_train, x2: x2_train})
        # print("Train accuracy: {:3.2f}%".format(acc_train*100.0))
        # 
        # acc_test = accuracy.eval(session= sess, feed_dict={inp: in_test, out: out_test, x1: x1_test, x2: x2_test})
        # print("Test accuracy:  {:3.2f}%".format(acc_test*100.0))
        # 
        # # Evaluate
        acc_eval = accuracy.eval(session= sess, feed_dict={inp: in_eval_matrix, out: out_eval_matrix, x1: x1_eval_matrix, x2: x2_eval_matrix})
        print("Eval accuracy:  {:3.2f}%".format(acc_eval*100.0))    
        acc_all += acc_eval

        # Save Dataframe with Accuracy over Iterations
        acc_iter['Accuracy'] = acc_iter['Accuracy'] + accuracy_data

# Mean Accuracy
print("Mean Eval accuracy:  {:3.2f}%".format((acc_all/n)*100.0))  

# Save Mean Accuracy per Iteration
acc_iter['Accuracy'] = acc_iter['Accuracy'].divide(n)
pd.DataFrame(acc_iter['Accuracy']).to_hdf('df_acc_gru_mean.h5',key='df', mode='w')  


Eval accuracy:  95.49%
Eval accuracy:  95.49%
Eval accuracy:  95.42%
Eval accuracy:  95.46%
Eval accuracy:  95.39%
Eval accuracy:  95.48%
Eval accuracy:  95.43%
Eval accuracy:  95.48%
Eval accuracy:  95.36%
Eval accuracy:  95.52%
Eval accuracy:  95.40%
Eval accuracy:  95.43%
Eval accuracy:  95.51%
Eval accuracy:  95.51%
Eval accuracy:  95.48%
Eval accuracy:  95.46%
Eval accuracy:  95.47%
Eval accuracy:  95.42%
Eval accuracy:  95.46%
Eval accuracy:  95.50%
Mean Eval accuracy:  95.46%
